In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
            pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Installed OpenAI Whisper**

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

# Imported Whisper

In [ ]:
import whisper


In [ ]:
# import whisper

# model = whisper.load_model("large")
# result = model.transcribe("/kaggle/input/eval-data/GV_Eval_3h/Audio/01-00004-02.mp3",language='hi')
# print(result["text"])

In [ ]:
# 

# **Installed asr-evaluation for wer evaluation**

In [ ]:
!pip install asr-evaluation

# Loaded the large model as it performed best on hindi data

In [ ]:
model = whisper.load_model("large")

getting all the file paths using glob module

In [ ]:
import glob
folder_path = "/kaggle/input/eval-data/GV_Eval_3h/Audio"
files = glob.glob(os.path.join(folder_path, "*"))

In [ ]:
files[0][-15:-4]

# created a predicted.txt file to store the predicted results, it can be found in predicted-data dataset

In [ ]:
for i in range(len(files)):
    with open('/kaggle/working/predicted.txt','a') as f:
        result = model.transcribe(files[i],language='hi') #the language is set to hindi for hindi transcription
        f.write(files[i][-15:-4] + result['text'] + '\n')
        print(files[i][-15:-4] + result['text'])
        print(i)
        del result
        
        

# created a preprocess function for text preprocessing

In [ ]:
def preprocess(text):
    punc = "!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~"
    return ''.join(ch for ch in text if ch not in punc).strip()

# created 2 seperate dictionary for real and predicted data because while loading the files using glob the file sequence was lost

# both real and pred dict contains ids as key and text as value

In [ ]:
real_dict={}

with open("/kaggle/input/eval-data/GV_Eval_3h/text", "r") as file:
  lines = file.readlines()  
  for line in lines:
    real_dict[line.split()[0]]=preprocess(line[12:])

In [ ]:
predicted_dict={}

with open("/kaggle/input/predicted-data/predicted.txt", "r") as file:
  lines = file.readlines()  
  for line in lines:
    predicted_dict[line.split()[0]]=preprocess(line[12:])

# Created 2 new files from both dictionaries in ordered-data dataset. These file are sequentially aligned based on the ids.

In [ ]:
for real_key in list(real_dict.keys()):
    for pred_key in list(predicted_dict.keys()):
        if real_key==pred_key:
            with open('/kaggle/working/new_real.txt','a') as f:
                f.write(real_dict[real_key] + '\n')
            with open('/kaggle/working/new_pred.txt','a') as f:
                f.write(predicted_dict[pred_key] + '\n')
    

# WER evaluation

In [ ]:
!wer /kaggle/input/oredered-data/new_real.txt  /kaggle/input/oredered-data/new_pred.txt

# Detailed WER evaluation

In [ ]:
!wer -r -c -p /kaggle/input/oredered-data/new_real.txt  /kaggle/input/oredered-data/new_pred.txt